In [1]:
### Imports

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def generate_sequence(list_of_words, sequence_length):
    res = []
    for i in range(len(list_of_words)-sequence_length):
        res.append((list_of_words[i:i+sequence_length],list_of_words[i+sequence_length]))
    return res

def splitter(database, fraction, num_words):
    
    training = [x for x in database['tokens']]
    labels = [x for x in database['label_tokens']]

    X_train = np.array(training[:int(fraction*len(training))])
    X_test = np.array(training[int(fraction*len(training)):])

    y_train_base = np.array(labels)[:int(fraction*len(labels))]
    y_test_base = np.array(labels)[int(fraction*len(labels)):]

    y_train = np.zeros((len(y_train_base), num_words), dtype=np.int8)
    y_test = np.zeros((len(y_test_base), num_words), dtype=np.int8)

    # One hot encoding of labels
    for example_index, word_index in enumerate(y_train_base):
        y_train[example_index, word_index] = 1

    for example_index, word_index in enumerate(y_test_base):
        y_test[example_index, word_index] = 1

    print(f'The training sequence shape is {X_train.shape}, the training label shape is {y_train.shape}')
    print(f'The test sequence shape is {X_test.shape}, the test label shape is  {y_test.shape}')
    return X_train, X_test, y_train, y_test

In [3]:
books= []


delims = ' |\n|\ufeff|_'
punctuation = '.|,|;|:'
wrapping = r'\--(.--?)\--'
sequence_length = 50

for i in range(1,11):
    with open(f'../Data/ABT/{i}.txt', mode="r", encoding="utf-8") as file:
        contents = file.read().lower()
        contents = re.sub(r'[^\w\s]', '', contents)
        contents = re.split(delims, contents)
        books.append([x for x in contents if x != ''])

for ind, book in enumerate(books):
    books[ind] = generate_sequence(book, sequence_length=sequence_length)

master = []

for book in books:
    for sequence in book:
        master.append(sequence)

master_frame = pd.DataFrame(master, columns=['sequence','label'])
master_frame.sample(10)

,sequence,label
600218,"[shown, and, explained, and, each, was, wonder...",a
263459,"[the, stranger, said, quietly, i, saw, and, he...",made
63108,"[bigger, than, a, large, sewerpipe, and, found...",wormed
684421,"[whispered, with, enthusiasm, and, belief, of,...",he
501893,"[the, limited, personality, he, had, been, so,...",result
180928,"[lot, welcome, the, lot, and, thushe, could, n...",in
489878,"[the, practical, results, obtainable, by, soun...",are
589941,"[for, one, another, it, was, the, mating, of, ...",in
640624,"[some, ethereal, bird, of, fire, rising, into,...",could
167655,"[a, voice, that, held, a, note, of, softness, ...",devonham


In [4]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(master_frame['sequence'])
master_frame['tokens'] = tokenizer.texts_to_sequences(master_frame['sequence'])
master_frame['label_tokens'] = tokenizer.texts_to_sequences(master_frame['label'])

word_lexicon = tokenizer.word_index
word_index = tokenizer.index_word
num_words = len(word_lexicon) + 1
word_counts = tokenizer.word_counts
sorted_counts = dict(sorted(dict(word_counts).items(), key=lambda item: item[1],reverse=True))

master_frame.sample(5)

,sequence,label,tokens,label_tokens
332083,"[inner, catastrophe, he, dreaded, while, desir...",found,"[380, 3581, 6, 2032, 130, 6006, 9, 15, 22, 54,...",[169]
332598,"[cause, unknown, was, the, scientific, verdict...",of,"[747, 857, 8, 1, 1954, 6595, 2, 38, 1730, 8210...",[3]
551636,"[came, wriggling, along, like, a, great, cat, ...",but,"[66, 8882, 299, 53, 4, 104, 751, 5, 9688, 28, ...",[20]
152322,"[in, the, private, sittingroom, levallon, seat...",been,"[7, 1, 1259, 2920, 328, 2502, 7, 38, 1881, 35,...",[71]
438894,"[me, with, a, vague, unrest, unsettled, seekin...",that,"[28, 13, 4, 1349, 4423, 7752, 1721, 5, 2710, 7...",[11]


In [7]:
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dropout, Bidirectional, Dense
from keras.models import Sequential, load_model

callback = tf.keras.callbacks.ModelCheckpoint(
                                                filepath='../Data/weights_RNN.h5',
                                                save_weights_only=True,
                                                monitor='val_accuracy',
                                                mode='max',
                                                save_best_only=True)

LSTM_cells = 64

model = Sequential()
model.add(Embedding(num_words, 300, input_length=50))
model.add(LSTM(LSTM_cells,return_sequences=True,dropout=0.1))
model.add(LSTM(LSTM_cells,return_sequences=True,dropout=0.1))
model.add(Bidirectional(LSTM(LSTM_cells,return_sequences=False,dropout=0.1)))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_words, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.save('../Data/RNN_model.h5')

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 300)           7485600   
                                                                 
 lstm_3 (LSTM)               (None, 50, 64)            93440     
                                                                 
 lstm_4 (LSTM)               (None, 50, 64)            33024     
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 300)               38700     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                      

In [8]:
BATCH_SIZE = 64
steps_per_epoch = 40
epochs = 40

step = 0.2
cut_list = step * np.arange(5)

for start, end in zip(cut_list, cut_list+step):
    
        temp_frame = master_frame.iloc[int(start*len(master_frame)):int(end*len(master_frame))+1]

        X_train, X_test, y_train, y_test = splitter(temp_frame,0.9, num_words)

        model = load_model('../Data/RNN_model.h5')

        model.fit(X_train, y_train, batch_size=64,
                steps_per_epoch=steps_per_epoch,
                epochs=epochs,
                callbacks=callback,
                validation_data=(X_test,y_test),
                validation_steps=10)

        model.save('../Data/RNN_model.h5')

The training sequence shape is (129563, 50), the training label shape is (129563, 24952)
The test sequence shape is (14396, 50), the test label shape is  (14396, 24952)
Epoch 1/40
40/40 [==============================] - 15s 121ms/step - loss: 8.7717 - accuracy: 0.0598 - val_loss: 7.9453 - val_accuracy: 0.0469
Epoch 2/40
40/40 [==============================] - 1s 36ms/step - loss: 7.4245 - accuracy: 0.0590 - val_loss: 7.7670 - val_accuracy: 0.0469
Epoch 3/40
40/40 [==============================] - 1s 34ms/step - loss: 7.2996 - accuracy: 0.0551 - val_loss: 7.6642 - val_accuracy: 0.0469
Epoch 4/40
40/40 [==============================] - 1s 36ms/step - loss: 7.2237 - accuracy: 0.0617 - val_loss: 7.6182 - val_accuracy: 0.0469
Epoch 5/40
40/40 [==============================] - 2s 38ms/step - loss: 7.0598 - accuracy: 0.0645 - val_loss: 7.5614 - val_accuracy: 0.0469
Epoch 6/40
40/40 [==============================] - 1s 35ms/step - loss: 7.1903 - accuracy: 0.0520 - val_loss: 7.6230 - val_

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.